In [ ]:
import cv2
from ultralytics import YOLO
import numpy as np

In [ ]:
# Load segmentation model
modelSeg = YOLO("yolov8l-seg.pt")


In [ ]:
def calcMask(result):
    raw = result.masks[0].cpu().data.numpy().transpose(1, 2, 0)
    rgb = cv2.merge((raw, raw, raw))
    h2,w2, _ =result.orig_img.shape
    mask = cv2.resize(rgb, (w2, h2))

    masked = cv2.blur(mask, (10, 10))

    lower_black = np.array([0,0,0])
    upper_black = np.array([0,0,0])

    mask = cv2.inRange(masked, lower_black, upper_black) # Smoothing the mask. replace masked to mask to have sharper mask
    mask=cv2.bitwise_not(mask)
    return mask

In [ ]:

cap = cv2.VideoCapture(0)
assert cap.isOpened(), "Error reading video file"

while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    results=modelSeg.predict(im0, device="mps")


    if(results[0].masks is not None):
        mask = calcMask(results[0])
        cv2.imshow("mask", mask)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

cap.release()
cv2.destroyAllWindows()